In [1]:
import os
import pandas as pd
import json

In [2]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

c:\Users\kimso\anaconda3\envs\test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 데이터 불러오기

#### 1. 가사

In [3]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

In [4]:
lyrics.head()

,index,id,title,singer,genres,lyrics,translated_lyrics,sentiment,sentiment_label
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay,2,분노
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐,5,당황
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루,0,기쁨
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼,0,기쁨
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변,2,분노


#### 2. 소설

In [ ]:
novel_final = pd.read_csv('./dataset/classified_novel1.csv')
data = novel_final.copy()

AttributeError: 'DataFrame' object has no attribute 'select'

In [136]:
data = data.sample(n=50000, random_state=42).reset_index(drop=True)

len(data)

50000

In [137]:
data.head()

,Column1,file_id,title,paragraph_id,text,labels,sentiment
0,347937,WARW1900000524,남도빨치산 6,WARW1900000524.1.2310,"""손들고 나왓!""",0.0,기쁨
1,570921,WARW1900000808,제갈공명 2-하늘로 치솟아 뜻을 펼치다,WARW1900000808.1.1898,"""그 점은 이 양이 생각해둔 바 있으니 과히 염려하지 마십시오. 틀림없이 연환계를 ...",0.0,기쁨
2,257278,WARW1900000461,890만번 주사위 던지기,WARW1900000461.1.504,"""102…103…104…….""",4.0,불안
3,702456,WARW1900001046,경찰청장 박전전 3,WARW1900001046.1.841,그 두 가지 사실이 무엇을 의미하는가?,1.0,슬픔
4,228104,WARW1900000425,퇴원,WARW1900000425.1.1765,"""어서 그냥 쇠새끼나 쫓아가보거라. 꼴짐은 내가 거둬 지고 가마.""",2.0,분노


### Preprocessing

In [138]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 0
None 포함 여부: 0


In [139]:
data['text'] = data['text'].fillna("")

In [140]:
print("NaN 개수:", data['text'].isna().sum())  # NaN 개수 확인
print("None 포함 여부:", data['text'].isnull().sum())  # None 개수 확인

NaN 개수: 0
None 포함 여부: 0


In [141]:
# 데이터셋 클래스 정의
class NovelsDataset(Dataset):
    def __init__(self, data, tokenizer, text_column="input_text", max_length=128):
        self.data = data.reset_index(drop=True)  # 인덱스 리셋 (중복 방지)
        self.tokenizer = tokenizer
        self.text_column = text_column
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.loc[idx, self.text_column] # input_text 가져오기

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        # labels 생성 (input_ids 복사 및 패딩 토큰을 -100으로 설정)
        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
        }

### Modeling

In [142]:
# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

# 모델 초기화
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.bos_token_id = tokenizer.bos_token_id

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [143]:
dataset = NovelsDataset(data, tokenizer, text_column="text")
dataloader = DataLoader(dataset, batch_size=2)

In [144]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    save_steps=200,
    save_total_limit=2,
    logging_steps=50,
    eval_strategy="epoch",
)

In [145]:
len(data)

50000

In [146]:
train_novels, eval_novels = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = NovelsDataset(train_novels, tokenizer, text_column="text")
eval_dataset = NovelsDataset(eval_novels, tokenizer, text_column="text")

In [147]:
len(train_novels)

45000

In [148]:
# Trainer에 데이터셋 전달
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 검증 데이터셋
)

In [149]:
sample = train_dataset[0]
print("Input IDs:", sample['input_ids'])
print("Attention Mask:", sample['attention_mask'])
print("Labels:", sample['labels'])

Input IDs: tensor([10063, 18331,  7177,  6995,  7162,  9298,  9436,  9126, 16148,  9536,
         9049, 11607, 24699, 40734, 23916, 11888, 11768,  9199, 13177,  8702,
         8062, 23775,   406,   377,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,    

In [150]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.530700,4.555839
2,4.071900,4.492127
3,3.581800,4.504992
4,2.946600,4.603178


TrainOutput(global_step=14060, training_loss=3.678667727778341, metrics={'train_runtime': 5246.2433, 'train_samples_per_second': 42.888, 'train_steps_per_second': 2.68, 'total_flos': 1.4692973543424e+16, 'train_loss': 3.678667727778341, 'epoch': 4.9984})

In [222]:
# 이야기 생성 함수
def generate_story(lyrics_input):
    # 입력 토큰화
    input_ids = tokenizer.encode(lyrics_input, return_tensors='pt').to(device)

    # 출력 생성
    output = model.generate(
        input_ids, 
        max_new_tokens=150, # 새로 생성할 토큰의 개수를 제한
        num_return_sequences=1, 
        temperature=0.8, 
        top_k=50, 
        top_p=0.9, 
        repetition_penalty=1.2, 
        do_sample=True  # 샘플링 활성화
    )

    # 입력 길이 추적
    input_length = input_ids.shape[1]

    # 생성된 토큰 중 입력 토큰 이후의 부분만 디코딩
    generated_tokens = output[0][input_length:]
    generated_story = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_story

In [223]:
lyrics1 = pd.read_csv("./dataset/label_result_song_short.csv")
lyrics2 = pd.read_csv("./dataset/translated_lyrics.csv")

lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])

In [224]:
# groupby와 agg를 활용해 변환
lyrics_input = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": list,  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
}).reset_index()

In [225]:
lyrics_input.head()

,id,title,singer,genres,lyrics,translated_lyrics
0,418168,희재,성시경,"발라드, 국내영화","[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ...","[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 ..."
1,418598,친구라도 될 걸 그랬어,거미 (GUMMY),R&B/Soul,"[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는...","[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는..."
2,711626,살다가,SG 워너비,발라드,"[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게...","[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게..."
3,1500196,내사람,SG 워너비,R&B/Soul,"[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도...","[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도..."
4,1854856,라라라,SG 워너비,발라드,"[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감...","[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감..."


In [226]:
i = 751
print(lyrics_input["title"][i])

내 이름 맑음


In [227]:
prompt = " ".join(lyrics_input["translated_lyrics"][i])

print(prompt)

어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 뭔가 들켜 버린 것 같아 표정을 보니 말이야 나도 티가 나버린 고백에 얼마나 놀랐는지 몰라 매일 치는 장난에도 두근댔고 오늘도 몇 번이고 떨렸지만 약속했어 날 안아줘 좀 알아줘 이건 꿈에서만 하기야  무심코 던진 니 말에 하루 종일 설레어 간직했다 아무도 못 보게 일기장에 적어 단단히 잠궜었는데 어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 꾹꾹 참고 또 꼭꼭 숨겨서 이제까지 잘해 왔잖아 그러다 고작 울음도 못 참고 괜찮다 말하며 두 눈은 퉁퉁 붓고 코맹맹이가 되어도 난 내일은 맑음 예전처럼 옆에서 밥 먹어도 우연히 눈이 살짝 마주쳐도 걱정 마 날 안아줘 아니 사랑해줘 이건 꿈에서만 하니까 무심코 던진 니 말에 하루 종일 설레어 간직했다 아무도 못 보게 꼬깃꼬깃 구겨 씹어 다 삼켰었는데 어쩌다 고작 그 마음도 못 참고 멍청하게 다 던졌는지 꾹꾹 참고 또 꼭꼭 숨겨서 이제까지 잘 해 왔잖아 그러다 고작 울음도 못 참고 괜찮다 말하며 두 눈은 퉁퉁 붓고 코맹맹이가 되어도 난 사실 나  아주 오래 울 것 같아 고작 친구도 못 되니까 툭툭 털고 활짝 웃을 만큼 나는 그리 강하지가 않아 그러다 고작 사랑이 뭐라고 괜찮다 말하는 날까지  꾹꾹 참고 또 일기나 쓰고 있어 나 내 이름 맑음


In [229]:
# 가사로 이야기 생성
generated_story = generate_story(prompt)
print("생성된 이야기:", generated_story)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# 생성된 이야기들을 저장할 리스트
generated_stories = []

# 651부터 750까지 반복하여 가사로 이야기 생성
for i in range(651, 751):
    # 각 가사 제목과 번역된 가사 추출
    title = lyrics_input["title"][i]
    prompt = " ".join(lyrics_input["translated_lyrics"][i])

    # 가사로 이야기 생성
    generated_story = generate_story(prompt)

    # 결과 저장
    generated_stories.append({"title": title, "generated_story": generated_story})

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(generated_stories)

# CSV로 저장
df.to_csv("generated_stories_651_750.csv", index=False)

print("저장 완료.")

In [ ]:
# 가사로 이야기 생성
# lyrics_input = ""
# generated_story = generate_story(lyrics_input, tag1="2", tag2="2")
# print("생성된 이야기:", generated_story)

생성된 이야기: 인지 이젠데 왜 내가 좋아하는지 그래 속상하기만 말예 퇴근거리는 게 참으론 정말이지 남이 나를 버리고 지금이 딱히 한숨 쉬었는지 알면서.... 이러면 금방엔 숨소리 질러 가며 소리 죽여 버릇한 기분이 좋아서 그냥 그러는 거냐? 이제 더러운 세상 물끄떡하고 예사로운 걸어서 그런 건가 못해~ 하고 사는데. 그만 이렇게 예쁜 척하는 소리가 나랑말루 오죽 나래 그래 가지 않고 그저 눈 감고 삐딱다리 놓으면 살그렁거니 이마에 이어두 못 하게 할까 봐놓고 사는 저러다 말고 다리 뻗고 가만히 들여다보면 예뻐한다ᆞ오래야 제자리 앉으며 맘
